In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install keras.utils
import keras.utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2634 sha256=11890645eb440051b63a5420458d93726670cc4af3437dfc39e4704cd6cac5f3
  Stored in directory: /root/.cache/pip/wheels/5c/c0/b3/0c332de4fd71f3733ea6d61697464b7ae4b2b5ff0300e6ca7a
Successfully built keras.utils


## Import Packages

In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, AveragePooling2D, BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.callbacks import Callback

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [5]:
No_pos= 62;
No_neg = 68;
pos_samples = np.zeros((No_pos, 80, 80))
neg_samples = np.zeros((No_neg, 80, 80))
#print(pos_samples)
#print(pos_samples[32,:,:].shape)

Read Data from file

In [6]:
index = 0;
for root, dirnames, filenames in os.walk('/content/drive/MyDrive/M.Phil/ADNI2/AD/'):
  for dirname in dirnames:
    path = ('/content/drive/MyDrive/M.Phil/ADNI2/AD/' + dirname + "/" + 'NormAdj_Matrix_WeightedN.csv')
    data = pd.read_csv(path, header=None)
    pos_samples[index, :, : ] = data
    print(index, dirname)
    index = index + 1
index = 0;
for root, dirnames, filenames in os.walk('/content/drive/MyDrive/M.Phil/ADNI2/CN/'):
  for dirname in dirnames:
    path = ('/content/drive/MyDrive/M.Phil/ADNI2/CN/' + dirname + "/" + 'NormAdj_Matrix_WeightedN.csv')
    #print(path)
    data = pd.read_csv(path, header=None)
    neg_samples[index, :, : ] = data
    print(index + 62, dirname)
    index = index + 1

0 007_S_4568
1 003_S_4136
2 005_S_4707
3 003_S_5165
4 003_S_4373
5 005_S_4910
6 005_S_5119
7 003_S_4892
8 005_S_5038
9 003_S_4152
10 016_S_5032
11 016_S_4887
12 021_S_4718
13 027_S_4962
14 029_S_4307
15 027_S_4964
16 027_S_4802
17 007_S_4911
18 021_S_4924
19 027_S_4938
20 098_S_4201
21 127_S_4500
22 109_S_4378
23 098_S_4215
24 094_S_4282
25 126_S_4686
26 126_S_4494
27 052_S_4959
28 094_S_4737
29 127_S_4749
30 127_S_5067
31 127_S_5056
32 127_S_4940
33 003_S_6833
34 006_S_6689
35 013_S_6768
36 016_S_6708
37 027_S_6648
38 027_S_6733
39 027_S_6849
40 032_S_6600
41 033_S_6705
42 033_S_6824
43 057_S_6746
44 094_S_6736
45 114_S_6347
46 114_S_6368
47 114_S_6595
48 116_S_6100
49 116_S_6543
50 126_S_6683
51 127_S_6433
52 129_S_6763
53 135_S_6284
54 135_S_6389
55 135_S_6545
56 168_S_6735
57 168_S_6754
58 168_S_6827
59 168_S_6828
60 168_S_6843
61 301_S_6592
62 003_S_4441
63 003_S_4350
64 003_S_4555
65 003_S_4644
66 003_S_4081
67 003_S_4288
68 003_S_4119
69 029_S_4290
70 007_S_4516
71 003_S_4839
72

Data and Labeling

In [7]:

Data = np.concatenate((pos_samples, neg_samples), axis =0)
#print(data.shape)
Y_pos = np.ones(No_pos)
Y_neg = np.zeros(No_neg)
Label = np.concatenate((Y_pos, Y_neg), axis =0)
#train_data, test_data,train_label, test_label = train_test_split(Data, Label, test_size=0.2, shuffle=True )
print(Data.shape)
# print(Label.shape)
Data = Data.reshape(Data.shape[0], Data.shape[1], Data.shape[2], 1)
print(Label.dtype)

(130, 80, 80)
float64


In [8]:
print(Data.shape)
Data = np.delete(Data,[94, 95, 104, 108],axis=0)
print(Data.shape)
print(Label.shape)
# Label = np.delete(Label,delete_index)
# print(Label.shape)
# print(Label)

(130, 80, 80, 1)
(126, 80, 80, 1)
(130,)


In [9]:
print(Data.shape)
Data = np.delete(Data,[109,110,119, 125],axis=0)
print(Data.shape)
print(Label.shape)

(126, 80, 80, 1)
(122, 80, 80, 1)
(130,)




```
# This is formatted as code
```

Deep Learning Model

In [10]:
def create_model():
  model = Sequential()
  model.add(Conv2D(filters = 32, kernel_size = 3, padding='valid',activation = 'relu', strides=(1, 1),input_shape = (80,80,1)))
  model.add(AveragePooling2D(pool_size=(2, 2)))
  model.add(Conv2D(filters = 64, kernel_size = 3, strides=(1, 1),padding='valid', activation = 'relu'))
  model.add(AveragePooling2D(pool_size=(2, 2)))
  model.add(Conv2D(filters = 128, kernel_size = 3, strides=(1, 1),padding='valid', activation = 'relu'))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(2, activation='softmax'))
  model.summary()
  opt = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [11]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = 3, padding='valid',activation = 'relu', strides=(1, 1),input_shape = (80,80,1)))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters = 64, kernel_size = 3, strides=(1, 1),padding='valid', activation = 'relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters = 128, kernel_size = 3, strides=(1, 1),padding='valid', activation = 'relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 78, 78, 32)        320       
                                                                 
 average_pooling2d (AverageP  (None, 39, 39, 32)       0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 37, 37, 64)        18496     
                                                                 
 average_pooling2d_1 (Averag  (None, 18, 18, 64)       0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 flatten (Flatten)           (None, 32768)             0

In [12]:
def train_and_evaluate__model(model, data_train, labels_train, data_test, labels_test):
  # callback = [TerminateOnBaseline(monitor='accuracy', baseline=1)]
  # callback = myCallback()
  print("Train Labels",labels_train)
  model.fit(x=data_train, y=labels_train, batch_size=64, epochs=100)
  prediticted_classes = model.predict(data_test, batch_size=4)
  print("Predict Classes", prediticted_classes)
  print("Evaluate on test data")
  results = model.evaluate(data_test, labels_test, verbose=0)
  print("test loss, test acc:", results)
  #test_labels = test_graphs.values
  #print(labels_test)
  #print('\nClassification Report\n')
  #print(classification_report(labels_test, prediticted_classes,  target_names=['Class 0', 'Class 1']))
  #print(confusion_matrix(labels_test, prediticted_classes))
  return results[1]

K-Fold

In [17]:
splits = 5
accuracy = np.zeros(splits)
index = 0;
kf = KFold(n_splits=splits, shuffle=True)
for train_index, test_index in kf.split(Data):
  print("TRAIN:", train_index, "TEST:", test_index)
  print("Test label",Label[test_index] )
  keras.backend.clear_session()
  model = create_model()
  accuracy[index] = train_and_evaluate__model(model, Data[train_index], to_categorical(Label[train_index]), Data[test_index], to_categorical(Label[test_index]))
  from sklearn.metrics import classification_report
  from sklearn.metrics import confusion_matrix
  prediticted_classes = model.predict(Data[test_index], batch_size=4)
  print('\nClassification Report\n')
  text_labels = Label[test_index]
  print(text_labels)
  print(text_labels.shape)
  preditiction = np.argmax(prediticted_classes, axis=1)
  print(preditiction)
  print(preditiction.shape)
  print(classification_report(text_labels, preditiction, target_names=['Class 0', 'Class 1']))
  print(confusion_matrix(text_labels, preditiction))
  index+=1
print(accuracy)
print("Average is:", sum(accuracy)/5.0)


TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  16  17  18  19
  20  21  22  25  26  27  28  30  31  32  33  34  35  36  38  40  41  42
  43  44  45  46  47  48  49  51  52  53  54  55  56  58  59  60  61  62
  63  64  65  66  68  69  70  75  76  78  79  80  82  83  84  85  86  87
  88  89  90  92  94  95  96  97  98  99 100 101 102 103 104 105 106 109
 110 111 116 117 119 120 121] TEST: [ 14  15  23  24  29  37  39  50  57  67  71  72  73  74  77  81  91  93
 107 108 112 113 114 115 118]
Test label [1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 78, 78, 32)        320       
                                                                 
 average_pooling2d (AverageP  (None, 39, 39, 32)       0         
 ooling2D)                                        

*Clear* Model

In [ ]:
keras.backend.clear_session()